# Ataques ao RSA

O criptosistema RSA baseia-se na dificuldade em determinar $p$ e $q$ dado um inteiro $N$, tal que $N=p*q$ com $p$ e $q$ primos.

Pretende-se implementar os ataques à inversão da chave pública e inversão do criptograma, usando redução de bases.  Verificando para que gama de valores dos parâmetros $N,q$ esses ataques são viáveis.

## Teorema (CopperSmith, Howgrave-Graham,May)

Dado um inteiro $N>1$ e um polinómio mónico $f \in \mathbb{Z}[x]$, para todo o parâmetro racional $0<\beta\le 1$, é possivel determinar em tempo polinomial, todos os inteiros $w$ tais que:
$|w| \le N^{\beta ^2 /d}$ e $mdc(f(w),N) \ge N^{\beta}$.

A importância do algoritmo de CopperSmith e, de forma geral, dos algoritmos de redução de bases em reticulados, provém da sua aplicabilidade em ataques a algumas das técnicas mais usadas, nomeadamente ao RSA.

## Invertibilidade do criptograma RSA
No início do uso do RSA além de não se usar qualquer forma de *padding* aleatória, também prevalecia o mau hábito de usar como chave pública um número primo pequeno, sendo frequente usar-se, para cifrar, um expoente público $d = 3$. Consequentemente, o criptograma relacionava-se com o *plaintext* numa relação da forma:
$$c \equiv m^d mod N,$$
sendo N o módulo RSA e $c,m \in \mathbb{Z}_{N}^{*}$ o ciptograma e o *plaintext*, respetivamente.

Um **ataque por inversão de criptograma** resolve o problema: $$N,d,c\leadsto m,$$
sem passar pelo conhecimento da chave pública.
Para usar o **algoritmo de CopperSmith** vamos alterá-lo ligeiramente. Considere-se: $$m = m' + w$$ e o seguinte problema:
$$N,d,c,m' \leadsto w: (w+m')^d \equiv c \ mod N.$$

Isto é, conhecendo uma parte do *plaintext*, procura-se determinar a diferença: $w = m -m'$, entre o *plaintext* que se pretende conhcer e a parte que já é conhecida.

Assim o algoritmo de CopperSmith é, assim, aplicado ao polinómio: $$f(x) \equiv (x+m')^d -c$$
e o que se pretende determinar é a raiz modular deste $f$, i.e., pretende-se clacular $w$ que verifique: $$f(w) \equiv 0 \ mod \ N.$$

As condições do algoritmo exigem que $w$ seja limitada a um valor $X \le N^{\beta^2/d}.$

Usando a versão do algoritmo $\beta = 1$ e considerando que o ataque se faz em relação à má chave pública, $d = 3$, então será $X \le N^{1/3}$. Em termos de tamanh, X, que é o tamanho da icógnita w, tem um terço dos *bits* de N.

Com estes parâmetros, o algoritmo de CopperSmith calcula as raizes modulares do polinómio $$f(x) \equiv (x+m')^d-c.$$
A raiz $w$, que verifica $f(w) \equiv 0 \ mod \ N$, adicionada à parte conhecida $m'$, produz todo o *plaintext* m.

In [60]:
import time

debug = True

def matrix_overview(B, bound):
    '''
        To display matrix picture with 0 and X
    '''
    for i in range(B.dimensions()[0]):
        a = ('%02d ' % i)
        for j in range(B.dimensions()[1]):
            a += '0' if B[i,j] == 0 else 'X'
            a += ' '
        if B[i, i] >= bound:
            a += '~'
        print a

def coppersmith(pol, mod, beta, m, t, X):
    '''
    Coppersmith, Howgrave-Graham, May
    Finds a solution if:
    * b|mod, b >= mod^beta , 0 < beta <= 1
    * |x| < X
    '''
    d = pol.degree()
    n = d * m + t
    if not 0 < beta <= 1:
        raise ValueError("beta should belongs in (0, 1]")
    if not pol.is_monic():
        raise ArithmeticError("The polynomial must be monic!")
    # calculate bounds and display them
    """
    * we want to find g(x) such that ||g(X)|| <= b^m / sqrt(n)
    * we know LLL will give us a short vector v such that: ||v|| <= 2^((n - 1)/4) * det(L)^(1/n)
    * we will use that vector as a coefficient vector for our g(x)
    * so we want to satisfy: 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n)
    * so we can obtain ||v|| < N^(beta*m) / sqrt(n) <= b^m / sqrt(n)
    (it's important to use N because we might not know b)
    """
    if debug:
        # t optimized?
        print "\n# Optimized t?\n"
        print "we want X^(n-1) < N^(beta*m) so that each vector is helpful"
        cond1 = RR(X^(n-1))
        print "* X^(n-1) = ", cond1
        cond2 = pow(mod, beta*m)
        print "* N^(beta*m) = ", cond2
        print "* X^(n-1) < N^(beta*m) \n-> GOOD" if cond1 < cond2 else "* X^(n-1) >= N^(beta*m) \n-> BAD"

        # bound for X
        print "\n# X bound respected?\n"
        print "we want X <= N^(((2*beta*m)/(n-1)) - ((delta*m*(m+1))/(n*(n-1)))) / 2 = M"
        print "* X =", X
        cond2 = RR(mod^(((2*beta*m)/(n-1)) - ((d*m*(m+1))/(n*(n-1)))) / 2)
        print "* M =", cond2
        print "* X <= M \n-> GOOD" if X <= cond2 else "* X > M \n-> BAD"

        # solution possible?
        print "\n# Solutions possible?\n"
        detL = RR(mod^(d * m * (m + 1) / 2) * X^(n * (n - 1) / 2))
        print "we can find a solution if 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n)"
        cond1 = RR(2^((n - 1)/4) * detL^(1/n))
        print "* 2^((n - 1)/4) * det(L)^(1/n) = ", cond1
        cond2 = RR(mod^(beta*m) / sqrt(n))
        print "* N^(beta*m) / sqrt(n) = ", cond2
        print "* 2^((n - 1)/4) * det(L)^(1/n) < N^(beta*m) / sqrt(n) \n-> SOLUTION WILL BE FOUND" if cond1 < cond2 else "* 2^((n - 1)/4) * det(L)^(1/n) >= N^(beta*m) / sqroot(n) \n-> NO SOLUTIONS MIGHT BE FOUND"

        # warning about X
        print "\n# Note that no solutions will be found if you don't respect:\n* |root| < X \n* b >= mod^beta\n"

    # change ring of pol and x
    polZ = pol.change_ring(ZZ)
    x = polZ.parent().gen()

    # compute polynomials
    g = []
    for i in range(m):
        for j in range(d):
            g.append((x * X)**j * mod**(m - i) * polZ(x * X)**i)
    for i in range(t):
        g.append((x * X)**i * polZ(x * X)**m)
    # construct latice B
    B = Matrix(ZZ, n)

    for i in range(n):
        for j in range(i+1):
            B[i, j] = g[i][j]
    if debug:
        matrix_overview(B, mod^m)
    B = B.LLL()
    # transform shortest vector in polynomial
    new_pol = 0
    for i in range(n):
        new_pol += x**i * B[0, i] / X**i
    # factor polynomial
    potential_roots = new_pol.roots()
    print "potential roots:", potential_roots
    # test roots
    roots = []
    for root in potential_roots:
        if root[0].is_integer():
            result = polZ(ZZ(root[0]))
            if gcd(mod, result) >= mod^beta:
                roots.append(ZZ(root[0]))
    return roots

In [61]:
print "//TEST//"

# RSA gen options (for the demo)
length_N = 1024  # size of the modulus
Kbits = 200      # size of the root
e = 3

# RSA gen (for the demo)
p = next_prime(2^int(round(length_N/2)))
q = next_prime(p)
N = p*q
ZmodN = Zmod(N);

# Create problem (for the demo)
K = ZZ.random_element(0, 2^Kbits)
Kdigits = K.digits(2)
M = [0]*Kbits + [1]*(length_N-Kbits); 
for i in range(len(Kdigits)):
    M[i] = Kdigits[i]
M = ZZ(M, 2)
C = ZmodN(M)^e

# Problem to equation (default)
P.<x> = PolynomialRing(ZmodN) #, implementation='NTL')
pol = (2^length_N - 2^Kbits + x)^e - C
d = pol.degree()

# Tweak those
beta = 1                                # b = N
epsilon = beta / 7                      # <= beta / 7
m = ceil(beta**2 / (d * epsilon))     # optimized value
t = floor(d * m * ((1/beta) - 1))    # optimized value
X = ceil(N**((beta**2/d) - epsilon))  # optimized value

# Coppersmith
start_time = time.time()
roots = coppersmith(pol, N, beta, m, t, X)

# output
print "\n# Solutions"
print "we want to find:",str(K)
print "we found:", str(roots)
print("in: %s seconds " % (time.time() - start_time))
print "\n"


//TEST//

# Optimized t?

we want X^(n-1) < N^(beta*m) so that each vector is helpful
* X^(n-1) =  5.26588450218277e469
* N^(beta*m) =  580960599536995806285950253330457437068697517636289523666148615228720373099711022573733604453311840725132615775498051744399052959454004712166288567218731837917089309138077931442122663713827534947029085316078452109665076499295326189203315508896470177354894140206439368479494675021108673629067157710453365964205711891714358440235842576345326856451291974202149556285680423718183094411859765263311077786050139575873400938210327751934796551227031831165077574060924571237598918752316257591089340137187066264183032068116642715033543814526336277062599577298105130865261347116571168133452984162663858268683943333726214301078023343862115325884086012061922836783087670464394234214189467644679504484114665996977282138358225609759740339015219683889811130249030757999762861745498961821675531013376918357608645939033771355595906598632573717762385934706974308955238231143411647257

Note-se que utilizou-se como padrão beta sempre igual a 1 e considerou-se X o seu limite superior na raiz. Observou-se que quanto maior a parte desconhecida, maior o X deveria ser, e consequentemente mais tempo demora.